# Web Scrapping of Twitter using Twitter API & Website Scrapping using BeautifulSoup
- by  Madhvi Asthana, Atharva Tonape,

## Web Scrapping of website using BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import datetime
from google.colab import drive

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

#Creation of Modal Class

class PageContent:
    Title = ""
    CreatedDate = ""
    Author = ""
    Content = []


    # remount drive, forced if needed
drive.mount("/content/gdrive/", force_remount=True)
print("Stablished access to Google Drive")

    # initialize Drive path
DRIVE_PATH = "/content/gdrive/My Drive"

    # open Firebase credentials
with open(DRIVE_PATH + "/ie_course_2022_team09/ie_course/credentials/firebase_credentials.json") as f:
    credential = json.load(f)

firebase_credential = credentials.Certificate(credential)

    # create Firestore database instance
firebase_admin.initialize_app(firebase_credential)
db = firestore.client()
print("Stablished access to Firestore")

#This function will do the link extraction that involves pagination

def GetFeed(pageNo):
    url = "https://spaceflightnow.com/category/news-archive/page/" + str(pageNo)

    r = requests.get(url)
    htmlContent = r.content
    # print(htmlContent)

    soup = BeautifulSoup(htmlContent, "html.parser")

    feed = soup.find_all("h3", {"class": "entry-title mh-loop-title"})
    linkList = []
    for i in feed:
        link = i.find("a")
        linkList.append(link["href"])
    print("Page : " + str(pageNo) + " ,  Successfully Scraped , Total Links Found " + str(len(linkList)))
    return linkList


#This function will do the data extraction and assign the values to the assigned keys

def GetPageContent(link):


    r = requests.get(link)
    htmlContent = r.content
    # print(htmlContent)

    soup = BeautifulSoup(htmlContent, "html.parser")
    content = PageContent()

    content.Title = soup.find("h1" , {"class" : "entry-title"}).text



    date = soup.find("span" , {"class" : "entry-meta-date updated"}).text

    format = '%B %d, %Y'  # The format
    datetime_str = datetime.datetime.strptime(date, format)
    content.CreatedDate = date

    content.Author = soup.find("span" , {"class" : "entry-meta-author vcard"}).text
    content.Content = soup.find("div" , {"class" : "entry-content clearfix"}).text
    import re
    cleanString = re.sub('\W+','', content.Title )
    print("\n+ Saving Post Data")
    print("+ Saving : {0}".format(content.Title))
    print(link)
    with open("/content/gdrive/My Drive/ie_course_2022_team09/ie_course/retrieved_data/{0}.json".format(cleanString), "w", encoding = "utf-8") as f:
        json.dump(content.__dict__,f, ensure_ascii = False, indent = 2)
    print("+ Saved in Google Drive")
    db.collection("WebScrappedPosts").document(cleanString).set(content.__dict__)
    print("+ Saved in FireStore\n")


#Main Function

if __name__ == "__main__":
    startPage = int(input("page_from "))
    endPage = int(input( "page_to "))
    listOfLinks = []
    for i in range(startPage,endPage+1):
        listOfLinks=(GetFeed(i))
        for i in listOfLinks:
          contenJson = GetPageContent(i)

    print("All Links Scraped  Successfully Scraper , Total Links Found " + str(len(listOfLinks)))








Mounted at /content/gdrive/
Stablished access to Google Drive
Stablished access to Firestore
page_from 81
page_to 82
Page : 81 ,  Successfully Scraped , Total Links Found 10

+ Saving Post Data
+ Saving : 
			For the first time, NASA is a spectator for a U.S. crew mission to low Earth orbit		
https://spaceflightnow.com/2021/09/14/for-the-first-time-nasa-is-a-spectator-for-a-u-s-crew-mission-to-low-earth-orbit/
+ Saved in Google Drive
+ Saved in FireStore


+ Saving Post Data
+ Saving : 
			Live coverage: Soyuz rocket launches with more OneWeb satellites		
https://spaceflightnow.com/2021/09/14/soyuz-oneweb-10-mission-status-center/
+ Saved in Google Drive
+ Saved in FireStore


+ Saving Post Data
+ Saving : 
			Russian rocket ready for launch with 34 more Florida-built OneWeb satellites		
https://spaceflightnow.com/2021/09/14/russian-rocket-ready-for-launch-with-34-more-florida-built-oneweb-satellites/
+ Saved in Google Drive
+ Saved in FireStore


+ Saving Post Data
+ Saving : 
			Spac

# Tweets Scrapping using API

In [ ]:
pip install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import tweepy
import csv
import re
import json
from datetime import datetime, timedelta


# Consumer keys and access tokens, used for OAuth


consumer_key = 'PRuWCZ8A0oEez4Pjo9zVbuoqR'
consumer_secret = 'mlG2dGIXRO1MAvkOusFb5IRmy2DilvFTCetI1oIkKJcTzaOktm'
access_token = '1490489563-WZUYcvfZahsxhuKDTL7nAJ0N6q0X1u0hCdaND47'
access_token_secret = 'mxQgPNHjAuQeOozjNPHVUAA0it9lbFk5bPLcbVHETKQSe'

# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Creation of the actual interface, using authentication
api = tweepy.API(auth, wait_on_rate_limit=True)



# creating a dictionary

tweets ={"id":[], "screen_name": [],"author_id" :[] ,"created_at":[],"text":[],"conversation_id":[],"lang":[]}

#taking username from users into a list

username = ['NASA','SpaceX']

#taking a variable to loop inside the list
user_cnt =0


while user_cnt != 2:
  # Main loop has started
  print (" Started")
  for year in range (2022, 2023):
    # looping between months
    for month in range(1 , 2):
      for status in tweepy.Cursor(api.user_timeline, screen_name=username[user_cnt], tweet_mode='extended').items():
        # There are three conditions here for months.
        if month %2 ==0:
          # For the month of February
          if month == 2:
            if status.created_at < datetime(year, month, 28, 1, 1, 1) and status.created_at > datetime(year, month, 1, 1, 1, 1):
              tweets["id"].append(status.id)
              tweets["screen_name"].append(username[user_cnt])
              tweets["author_id"].append(status.id_str)
              tweets["created_at"].append(status.created_at.strftime("%Y-%m-%d %H:%M:%S"))
              tweets["text"].append(status.full_text)
              tweets["conversation_id"].append(status.in_reply_to_status_id )
              tweets["lang"].append(status.lang)
          # For the month of 30 days
          else:
            if status.created_at < datetime(year, month, 30, 1, 1, 1) and status.created_at > datetime(year, month, 1, 1, 1, 1):
              # tweets.append(
              #                     {
              #                         'Date' : status.created_at,
              #                         'Tweet' : status.full_text
              #                     }
              #                   )
              tweets["id"].append(status.id)
              tweets["screen_name"].append(username[user_cnt])
              tweets["author_id"].append(status.id_str)
              tweets["created_at"].append(status.created_at.strftime("%Y-%m-%d %H:%M:%S"))
              tweets["text"].append(status.full_text)
              tweets["conversation_id"].append(status.in_reply_to_status_id )
              tweets["lang"].append(status.lang)
        # For the month of 31 days
        else:
          if status.created_at < datetime(year, month, 31, 1, 1, 1) and status.created_at > datetime(year, month, 1, 1, 1, 1):
              # tweets.append(
              #                     {
              #                         'Date' : status.created_at,
              #                         'Tweet' : status.full_text
              #                     }
              #                   )
              tweets["id"].append(status.id)
              tweets["screen_name"].append(username[user_cnt])
              tweets["author_id"].append(status.id_str)
              tweets["created_at"].append(status.created_at.strftime("%Y-%m-%d %H:%M:%S"))
              tweets["text"].append(status.full_text)
              tweets["conversation_id"].append(status.in_reply_to_status_id )
              tweets["lang"].append(status.lang)
      # writing a json file
      with open(''+username[user_cnt]+' '+str(month)+'-'+str(year)+'.json', 'w', encoding='utf-8') as file:
        json.dump(tweets, file, indent=4, ensure_ascii='False', default=str)
      print(tweets)
      # Writing the output to Firestore
      doc_ref = db.collection(u'Twitter_Scrapped').document(u''+username[user_cnt]+' '+str(month)+'-'+str(year))
      doc_ref.set(tweets)
      # We will need to reintialize the dictionary as previous months output are assigned to it
      tweets ={"id":[], "screen_name": [],"author_id" :[] ,"created_at":[],"text":[],"conversation_id":[],"lang":[]}
      # To see in the terminal which month is going on
      print ("Month Done "+str(month)+ " Account: "+username[user_cnt])
  year = year + 1
  user_cnt = user_cnt +1





 Started
{'id': [1487906828038160387, 1487834446610980871, 1487817965361934337, 1487553233509924867, 1487496134738255874, 1487221981317632003, 1487186153795166212, 1487164219971801091, 1487157310380249090, 1487108790881337348, 1486847414636064777, 1486832366807101443, 1486818778386247681, 1486801355096395780, 1486797273765924867, 1486789432481140736, 1486718227824611333, 1486513947712831491, 1486462851560755208, 1486443893453463566, 1486434557121466376, 1486114021547851782, 1486077491747901440, 1486058587071954949, 1486048414626365456, 1485990262656667649, 1485752219379064841, 1485738545734107137, 1485737927703506944, 1485736018322108417, 1485719996542164996, 1485704121453879302, 1485695318524059648, 1485677058105778178, 1485325632947032064, 1485279268494008323, 1485269983395553289, 1484967289854337032, 1484932576515182594, 1484898357130809348, 1484653058042830850, 1484651375862685708, 1484647992070221828, 1484632275224612864, 1484607466881097735, 1484374623852503042, 14843739986369863